In [3]:
import pandas as pd
import os
import sys
from datetime import datetime, timedelta
from dotenv import load_dotenv

from cognite.client.data_classes import functions
from cognite.client.data_classes.functions import FunctionSchedulesList
from cognite.client.data_classes.functions import FunctionSchedule

from initialize import initialize_client
cdf_env = "dev"

# Set limit on function calls - don't think it's really necessary ...
func_limits = functions.FunctionsLimits(timeout_minutes=60, cpu_cores=0.25, memory_gb=1, runtimes=["py39"], response_size_mb=2)
client = initialize_client(cdf_env)

NameError: name '__file__' is not defined

In [5]:
import requests
url = "https://pi-web-api.akerbp.com/piwebapi/streams/F1DPGrFe7LeAxUu_IwydrHHT2wNzEIAAQVBQLVBJLVNFUlZFUlwwMiBBTU9TLkVBU1QgS0FNRUxFT04gRkxPVyBBTEFSTS5DMEZGNUVFNy1CQUZBLTQzRDMtQTQ5Qy02RTQ5Njg5QUVCQTY/recorded?startTime=*-7d&maxCount=2"
auth = ("vetnev", "test")
r = requests.get(url, auth=auth)

##### Need to store model parameters in file, and upload this to CDF to be able to extract trained parameters for next call to the Cognite Function (fine-tuning for next schedule and prediction for subsequent period)

In [32]:
# data[col]: prints pd.Series object
# data[[col]]: prints pd.DataFrame object

myfunc = client.functions.retrieve(external_id="cf_power-Template")
my_schedule_id = client.functions.schedules.list(
                name="VAL_11-LT-95107A:X.Value", function_id=myfunc.id).to_pandas().id[0]

scheduled_calls = myfunc.list_calls(schedule_id=my_schedule_id, limit=-1).to_pandas()
start_time = (datetime(2024, 1, 15, 15, 30) - datetime(1970, 1, 1, 0, 0)).total_seconds()*1000
end_time = (datetime(2024, 1, 15, 15, 45) - datetime(1970, 1, 1, 0, 0)).total_seconds()*1000

mask_start = scheduled_calls["scheduled_time"] >= start_time
mask_end = scheduled_calls["scheduled_time"] < end_time
myid = scheduled_calls[mask_start & mask_end]["id"].iloc[0]
myfunc.retrieve_call(id=myid).get_response()
myfunc.list_calls().to_pandas().id[0]
# test = client.functions.calls.retrieve(call_id=3005253751851002, function_id=84587311037983).get_response()

579500946117362

## Time series future prediction

In [ ]:
import pathlib
from sklearn.model_selection import BaseCrossValidator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import BayesianRidge

import itertools
import numpy as np
import pickle


In [2]:
ts_input_name = "VAL_11-LT-95107A:X.Value"
ts_list = client.time_series.list(name=ts_input_name).to_pandas()
ts_extid = ts_list["external_id"][0]

ts_95107A = client.time_series.data.retrieve(external_id=ts_extid,
                                             granularity="10m",
                                             aggregates="average",
                                             start=pd.to_datetime(datetime(2023,1,1)),
                                             end=pd.to_datetime(pd.Timestamp.now())).to_pandas()

ts_95107A = ts_95107A.rename(columns={ts_extid+"|average": ts_input_name})
ts_95107A.head()

c:\Users\vetnev\AppData\Local\anaconda3\envs\task1\Lib\site-packages\cognite\client\_api\datapoints.py:806: UserWarning: Your installation of 'protobuf' is missing compiled C binaries, and will run in pure-python mode, which causes datapoints fetching to be ~5x slower. To verify, set the environment variable `PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=cpp` before running (this will cause the code to fail). The easiest fix is probably to pin your 'protobuf' dependency to major version 4 (or higher), see: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates
  fetcher = select_dps_fetch_strategy(self, user_query=query)


,VAL_11-LT-95107A:X.Value
2023-01-01 00:00:00,65.737658
2023-01-01 00:10:00,65.781379
2023-01-01 00:20:00,65.729704
2023-01-01 00:30:00,65.749212
2023-01-01 00:40:00,65.773900


In [15]:
# Train-test splits
max_idx = len(ts_95107A)-1
train_test_split = 0.7
X_train = ts_95107A.iloc[:int(train_test_split*max_idx)] # for CV grid search
X_test = ts_95107A.iloc[int(train_test_split*max_idx):] # to test for best model found from CV grid search

In [ ]:
num_folds = 10
test_size = 100 # here: 100 10-min periods

# 1. Use expanding window feature from Anders, or manually divide train set into expanding windows
# 2. If manual, make an expanding window model Class so to give a valid model input to CV GridSearch
# 3. Run CV GridSearch, extract best model
# 4. use best model to predict data in test set (most recent period OR future period). If future period, change train set to cover all data until current date, then define test set to be from last datapoint and forward 100 10-min periods
# 5. Plot historic and future data, but highlight future data in another color

In [8]:
class ExpandingWindowCV(BaseCrossValidator):
    def __init__(self, n_splits, start_len, test_len):
        self.n_splits = n_splits
        self.start_len = start_len
        self.test_len = test_len

    def split(self, X, y=None, groups=None):
        N = len(X)
        ws = int(np.floor(N / self.n_splits) - 1)
        for i in range(self.n_splits):
            Xi_train = X[:self.start_len + i*ws]
            Xi_test = X[self.start_len+i*ws:self.start_len+i*ws+self.test_len]
            yield Xi_train, Xi_test


    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits


In [36]:
cv = ExpandingWindowCV(n_splits=5, start_len=500, test_len=100)
# a = exp.split(ts_95107A)

In [37]:
estimator = Pipeline([
                ("poly", PolynomialFeatures(degree=2, interaction_only=True)),
                ("scaler", StandardScaler()),
                ("regressor", BayesianRidge(fit_intercept=True))
            ])

In [38]:
parameter_grid = {
    "lambda_1": [1e-6, 1e-5],
    "alpha_1": [1e-6, 1e-5],
}

In [40]:
model = GridSearchCV(
    estimator=estimator,
    cv=cv,
    param_grid=parameter_grid,
    n_jobs=2,
    scoring=make_scorer(mean_squared_error, greater_is_better=False),
    return_train_score=True,
    refit=True,
    verbose=3
)

print("Total number of fits:", cv.get_n_splits(X_train) * len(list(itertools.product(*parameter_grid.values()))))

Total number of fits: 20


In [43]:
exp_dir = pathlib.Path("").absolute().joinpath("../data/experimental")
if not exp_dir.exists():
    exp_dir.mkdir()

In [ ]:
print("1 - Train from scratch.", flush=True)
print("2 - Load results from previous training.", flush=True)
choice = input("What do you want to do?")

if int(choice) == 1:
    model.fit(X_train, y_train)
    cv_results = pd.DataFrame(model.cv_results_).sort_values(by="rank_test_score", ascending=True)
    cv_results.to_csv(exp_dir.joinpath("cross_validation_results.csv"))
    with open(exp_dir.joinpath("best_model.pkl"), "wb") as f:
        pickle.dump(obj=model.best_estimator_, file=f)
    print(f">>> ({choice}) Model trained from scratch and cross validation results written to file.")
elif int(choice) == 2:
    cv_results = pd.read_csv(exp_dir.joinpath("cross_validation_results.csv"), index_col=0).sort_values(by="rank_test_score", ascending=True)
    print(f">>> ({choice}) Loaded cross validation results from file.")
else:
    raise RuntimeError("Invalid choice")